In [1]:
from pathlib import Path
from utils import (
    loads_cae_db,
    executes_query_duckdb
)

In [2]:
cae_path = Path(
    "../../kids_chances/data/raw/cae_history.txt"
)

destionation_db = Path("../data/cae_db.duckdb")
cae = loads_cae_db(cae_path, destionation_db)

In [3]:
query = """
select distinct(tipo_beneficiario) from cae limit 10000"""
executes_query_duckdb(query, cae)

,tipo_beneficiario
0,EGRESO
1,SIN CREDITO POR NO MATRICULA O POR SUSPENSION
2,NUEVO BENEFICIARIO
3,BENEFICIARIO RENOVANTE
4,DESERCION
5,GRATUIDAD


In [44]:
query_sum = """

SELECT id_2023 as id,
        genero_registrado as genero,
        dependencia_colegio as colegio,
        region_grupo_familiar as region,
        quintil_ingreso_grupo_familiar as quintil,
        COUNT(CASE WHEN tipo_beneficiario = 'NUEVO BENEFICIARIO' THEN tipo_beneficiario END)
            as total_prestamos,
        COUNT(DISTINCT(nombre_carrera || '-' || nombre_ies)) as cantidad_carreras,
        COUNT(CASE WHEN arancel_solicitado > 0 then arancel_solicitado END)
            as años_financiados,
        COUNT(DISTINCT(CASE WHEN tipo_beneficiario = 'EGRESO' then (tipo_beneficiario || nombre_carrera || nombre_ies) END))
            as total_egresos,
        COUNT(DISTINCT(CASE WHEN tipo_beneficiario = 'DESERCION' then (tipo_beneficiario || nombre_carrera || nombre_ies) END))
            as total_deserciones,
        ROUND(SUM(total_prestado_ajustado),0) as total_prestado

FROM cae
GROUP BY id, genero, colegio, region, quintil
"""

In [45]:
h = executes_query_duckdb(query_sum, cae)
h

,id,genero,colegio,region,quintil,total_prestamos,cantidad_carreras,años_financiados,total_egresos,total_deserciones,total_prestado
0,984.947,F,Particular subvencionado,REGION DE COQUIMBO,quintil 3,1,1,0,1,0,0.0
1,343.352,F,Municipal,REGION DEL LIB. B. O'HIGGINS,quintil 1 y 2,1,1,2,1,0,3141708.0
2,342.936,F,Particular subvencionado,REGION METROPOLITANA,quintil 1 y 2,1,1,5,1,0,7991294.0
3,975.952,F,Particular pagado,REGION METROPOLITANA,quintil 4,1,2,6,1,0,18019949.0
4,346.871,F,Estudiante de Colegio o Liceo Particular subve...,REGION DE VALPARAISO,quintil 1 y 2,1,1,3,1,0,4910634.0
...,...,...,...,...,...,...,...,...,...,...,...
1262369,1.232.026,F,Municipal DAEM,REGION DE ANTOFAGASTA,quintil 3,1,1,1,0,0,3000000.0
1262370,1.202.832,M,Municipal DAEM,REGION DE LOS LAGOS,quintil 1 y 2,1,1,1,0,0,2783710.0
1262371,1.206.826,M,Particular Subvencionado,REGION METROPOLITANA,quintil 4,1,1,1,0,0,3178954.0
1262372,1.193.276,M,Corporación Municipal,REGION DE COQUIMBO,quintil 4,1,1,1,0,0,2604666.0


In [37]:
cae.execute("""select * from cae limit 0""").df()

,id_2023,año_licitacion,año_operacion,genero_registrado,nacionalidad,region_grupo_familiar,quintil_ingreso_grupo_familiar,cambio_ies_carrera,tipo_beneficiario,nivel_estudio_al_ingreso,...,nombre_carrera,duracion_formal_carrera,contador_años_financiados,dependencia_colegio,arancel_solicitado,arancel_referencia,tipo_titulo,jornada,banco_administrador,total_prestado_ajustado
